In [76]:
import dask.dataframe as dd
import pandas as pd

proyeccion=dd.read_csv('Proyeccion.csv', dtype={'DEPARTAMENTOS': 'object'})


In [ ]:
exportacion=dd.read_csv('2022_exportacion.csv', dtype={'COD_MODALIDAD_PRECEDENTE': 'object','NIT_EXPORTADOR': 'object'}).compute()



In [ ]:
importacion=dd.read_csv('2022_importacion.csv', dtype={'CODIGO_MODALIDAD_DECLARAC': 'object',       'PROGRAMA_NUMERO': 'object',     'TIPO_IDENTIFICAC_DECLARAN': 'object',   'TIPO_IDENTIFICAC_EXPORTAD': 'object', 'TIPO_IDENTIFICAC_IMPORTAD': 'object','COD_CLASE_IMPORTADOR': 'object', 'DIGITO_VERIFI_NIT_DECLARA': 'object','DIGITO_VERIFI_NIT_IMPORTA': 'object','NUMERO_IDENTIFICAC_EXPORT': 'object', 'CIUDAD_PAIS_IMPORTADOR': 'object'}).compute()

In [ ]:
import pandas as pd
# convert column mes to datetime, all years are 2022
exportacion['Mes'] = exportacion['Mes'].apply(lambda x: "1-" + str(x) + "-2022")
importacion['Mes'] = importacion['Mes'].apply(lambda x: "1-" + str(x) + "-2022")


In [ ]:
exportacion['Mes'] = pd.to_datetime(exportacion['Mes'], format='%d-%m-%Y')
importacion['Mes'] = pd.to_datetime(importacion['Mes'], format='%d-%m-%Y')


In [ ]:
correlativa=pd.read_excel("TOTPART-v74-Dic22.xlsx", skiprows=2)


In [ ]:
correlativa.drop_duplicates(subset ="Subpartida Arancelaria", keep = 'first', inplace = True)

In [ ]:
pd.set_option('display.max_columns', None)
## merge column if string contains another string in correlativa
exportacion['SUBPARTIDA'] = exportacion['SUBPARTIDA'].apply(lambda x: x if x not in correlativa['Subpartida Arancelaria'].values else correlativa[correlativa['Subpartida Arancelaria'] == x]['Subpartida Arancelaria'].values[0])


In [ ]:
#merge
exportacion2=exportacion.merge(correlativa, left_on='SUBPARTIDA', right_on='Subpartida Arancelaria', how='left')


In [ ]:
exportacion2.head()

In [ ]:
del exportacion2['Subpartida Arancelaria']
del exportacion2['Año_y']


In [ ]:
exportacion2=exportacion2.rename(columns={'Año_x': 'Año'}, inplace=True)

In [ ]:
exportacion

In [ ]:
exportacion3=exportacion2.groupby(['Mes','REGION_DE_ORIGEN','SUBPARTIDA', 'CIIU Rev. 4 A.C (2021)', 'Descripción Arancelaria']).agg({'PESO_NETO_KGS': 'sum', 'VALOR_FOB_USD': 'sum'})

In [ ]:
exportacionATLANTICO=exportacion3.loc[exportacion3.index.get_level_values('REGION_DE_ORIGEN') == 'ATLANTICO']

In [ ]:
ciiu=pd.read_excel("/Users/lasagna/Library/CloudStorage/OneDrive-UniversidaddelNorte/Probarranquilla Projects/PIB proyectado/articles-380405_recurso_1.xls", skiprows=2)

In [ ]:
exportacionATLANTICO=exportacionATLANTICO.reset_index()


In [ ]:
exportacionATLANTICO["CIIU Rev. 4 A.C (2021)"]=exportacionATLANTICO["CIIU Rev. 4 A.C (2021)"].astype(str)
ciiu['Clase']=ciiu['Clase'].astype(str)
exportacionATLANTICO["CIIU Rev. 4 A.C (2021)"]=exportacionATLANTICO["CIIU Rev. 4 A.C (2021)"].apply(lambda x: x if x not in ciiu['Clase'].values else ciiu[ciiu['Clase'] == x]['Clase'].values[0])

In [ ]:
exportacionATLANTICO=exportacionATLANTICO.merge(ciiu, left_on='CIIU Rev. 4 A.C (2021)', right_on='Clase', how='left')


In [ ]:
exportacionATLANTICO.to_csv('exportacionATLANTICOporMes.csv')

In [ ]:
import pandas as pd
exportacionATLANTICO=pd.read_csv('exportacionATLANTICOporMes.csv')
exportacionATLANTICO


El presente informe tiene como objetivo ofrecer un panorama general de la economía del departamento del Atlántico, su desempeño sectorial frente a la economía globalizada y su contribución a las cuentas nacionales en algunos sectores clave.

## 1. Comercio exterior

A lo largo de los meses del 2022, las exportaciones del Atlántico han mantenido un nivel de dinamismo constante. Al analizar la distribución específica de los productos según la Clasificación Industrial Internacional Uniforme (CIIU), se puede observar que el producto más exportado en el departamento ha sido consistentemente aquel relacionado con la fabricación de productos metálicos para estructuras.



In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'iframe_connected'
#show in jupyterq
exportacionATLANTICOfixtext=exportacionATLANTICO.copy()
exportacionATLANTICOfixtext['Descripción'] = exportacionATLANTICOfixtext['Descripción'].apply(lambda x: x if len(str(x)) < 30 else str(x)[:50] + '...')

# recortar texto

fig = px.bar(exportacionATLANTICOfixtext, x="Mes", y="VALOR_FOB_USD", color='Descripción')
#add title


fig.update_layout(title_text='Exportaciones Atlántico por mes')


fig.show()



Sin embargo, al agrupar estos productos por sector, se puede observar que durante el año 2022, el sector que tuvo un desempeño sobresaliente fue el de fabricación de productos elaborados de metal, excepto maquinaria y equipo, seguido por el sector de fabricación de sustancias y productos químicos, que habitualmente es aquel con mayor peso en las exportaciones del departamento, pero que en esta ocasión se ubicó en segundo lugar.

In [ ]:
#show all rows
pd.set_option('display.max_rows', None)
#left the first 2 characters
exportacionATLANTICOsec=exportacionATLANTICO.copy()
exportacionATLANTICOsec["CIIU Rev. 4 A.C (2021)"]=exportacionATLANTICOsec["CIIU Rev. 4 A.C (2021)"].astype(str).str[:2]

exportacionATLANTICOsec=exportacionATLANTICOsec.groupby(['CIIU Rev. 4 A.C (2021)']).agg({'VALOR_FOB_USD': 'sum'}).sort_values(by='VALOR_FOB_USD', ascending=False)

exportacionATLANTICOsec=exportacionATLANTICOsec.reset_index()

ciiu=pd.read_excel("/Users/lasagna/Library/CloudStorage/OneDrive-UniversidaddelNorte/Probarranquilla Projects/PIB proyectado/articles-380405_recurso_1.xls", skiprows=2)

exportacionATLANTICOsec["CIIU Rev. 4 A.C (2021)"]=exportacionATLANTICOsec["CIIU Rev. 4 A.C (2021)"].astype(float)
ciiu['Clase']=ciiu['Clase'].astype(float)
exportacionATLANTICOsec=exportacionATLANTICOsec.merge(ciiu, left_on='CIIU Rev. 4 A.C (2021)', right_on='División', how='left')

#IF LESS THAN 1% OF TOTAL, GROUP IN OTHER
exportacionATLANTICOsec['Descripción'] = exportacionATLANTICOsec['Descripción'].apply(lambda x: 'Otros' if x not in exportacionATLANTICOsec[exportacionATLANTICOsec['VALOR_FOB_USD'] > exportacionATLANTICOsec['VALOR_FOB_USD'].sum()*0.01]['Descripción'].values else x)
# recortar texto
exportacionATLANTICOsec['Descripción'] = exportacionATLANTICOsec['Descripción'].apply(lambda x: x if len(str(x)) < 30 else str(x)[:30] + '...')
FIG=px.pie(exportacionATLANTICOsec, values='VALOR_FOB_USD', names='Descripción', title='Exportaciones Atlántico por CIIU')
FIG.show()






Si enfocamos nuestro análisis en la clasificación CIIU de químicos para el año 2022 a nivel nacional, podemos observar que el departamento de Bolívar es el que más exporta este tipo de productos, seguido por Antioquia y Atlántico. Este último departamento se destaca como uno de los más fuertes en este sector en todo el territorio nacional, representando el 12,1% de las exportaciones totales de productos químicos en 2022.


In [ ]:
# convert from horizontal to vertical to use in excel chart
exportacionATLANTICO["VALOR_FOB_USD"]=exportacionATLANTICO["VALOR_FOB_USD"].astype(int)

exportacionATLANTICO2=exportacionATLANTICO.pivot_table(index=['Mes'], columns=['Descripción Arancelaria'], values=['VALOR_FOB_USD'], aggfunc='sum')

# covert the cols only the description and mes
exportacionATLANTICO2.columns = exportacionATLANTICO2.columns.droplevel(0)


In [ ]:

# # Create a Pandas Excel writer using XlsxWriter as the engine.
# excel_file = 'stacked_column.xlsx'
# sheet_name = 'Sheet1'
# df=exportacionATLANTICO2
# writer = pd.ExcelWriter(excel_file, engine='xlsxwriter')
# df.to_excel(writer, sheet_name=sheet_name)

# workbook = writer.book
# worksheet = writer.sheets[sheet_name]

# chart = workbook.add_chart({'type': 'column', 'subtype': 'stacked'})

# # Configure the series of the chart from the dataframe data.
# for col_num, value in enumerate(df.columns.values):
#     chart.add_series({
#         'name':       [sheet_name, 0, col_num],
#         'categories': [sheet_name, 1, 0, len(df), 0],
#         'values':     [sheet_name, 1, col_num, len(df), col_num],
#     })
    

# # Configure the chart axes.
# chart.set_x_axis({'name': 'Mes'})
# chart.set_y_axis({'name': 'Valor FOB USD'})

# # Insert the chart into the worksheet.
# worksheet.insert_chart('D2', chart)

# # Close the Pandas Excel writer and output the Excel file.
# writer.save()



/var/folders/pl/v8kzknmx44v8w9mzp7wds7z40000gn/T/ipykernel_7698/1020491204.py:30: FutureWarning:

save is not part of the public API, usage can give unexpected results and will be removed in a future version



In [ ]:
# compare with other regions

exportacionall=exportacion2

exportacionall.head()

ciiu=pd.read_excel("/Users/lasagna/Library/CloudStorage/OneDrive-UniversidaddelNorte/Probarranquilla Projects/PIB proyectado/articles-380405_recurso_1.xls", skiprows=2)
exportacionall=exportacionall.merge(ciiu, left_on='CIIU Rev. 4 A.C (2021)', right_on='Clase', how='left')

exportacionall=exportacionall.groupby(['CIIU Rev. 4 A.C (2021)', 'REGION_DE_ORIGEN']).agg({'VALOR_FOB_USD': 'sum'}).sort_values(by='VALOR_FOB_USD', ascending=False)
# just the first 2 characters
exportacionall=exportacionall.reset_index()

exportacionall["CIIU Rev. 4 A.C (2021)"]=exportacionall["CIIU Rev. 4 A.C (2021)"].astype(str).str[:2]
exportacionall=exportacionall.reset_index()
exportacionall["CIIU Rev. 4 A.C (2021)"]=exportacionall["CIIU Rev. 4 A.C (2021)"].astype(float)
ciiu['Clase']=ciiu['Clase'].astype(float)
exportacionall=exportacionall.merge(ciiu, left_on='CIIU Rev. 4 A.C (2021)', right_on='División', how='left')


# just show Fabricación de sustancias y productos químicos

exportacionallquim=exportacionall[exportacionall['Descripción']=='Fabricación de sustancias y productos químicos']

exportacionallquim=exportacionallquim.groupby(['REGION_DE_ORIGEN']).agg({'VALOR_FOB_USD': 'sum'}).sort_values(by='VALOR_FOB_USD', ascending=False)
exportacionallquim=exportacionallquim.reset_index()
exportacionallquim.sort_values(by='VALOR_FOB_USD', ascending=False)
#otros
exportacionallquim['REGION_DE_ORIGEN'] = exportacionallquim['REGION_DE_ORIGEN'].apply(lambda x: 'Otros' if x not in exportacionallquim[exportacionallquim['VALOR_FOB_USD'] > exportacionallquim['VALOR_FOB_USD'].sum()*0.01]['REGION_DE_ORIGEN'].values else x)

fig=px.pie(exportacionallquim, values='VALOR_FOB_USD', names='REGION_DE_ORIGEN', title='Exportaciones por CIIU para quimicos')
fig.show()

Si nos enfocamos en el sector de metales, podemos observar que el departamento del Atlántico es el ganador absoluto en términos de participación de mercado, representando el 61,2% de las exportaciones totales de productos metálicos en 2022. Los departamentos de Antioquia y Bogotá D.C. les siguen en importancia, con un 12,7% y un 10,2% de participación en el mercado, respectivamente.

In [ ]:
exportacionallquim

In [ ]:
# #recreate the graph in excel

# # columns are REGION_DE_ORIGEN	VALOR_FOB_USD

# exportacionallquim2=exportacionallquim


# # Create a Pandas Excel writer using XlsxWriter as the engine.
# excel_file = 'pie_chart.xlsx'
# sheet_name = 'Sheet1'

# df=exportacionallquim2

# writer = pd.ExcelWriter(excel_file, engine='xlsxwriter')
# df.to_excel(writer, sheet_name=sheet_name)

# workbook = writer.book
# worksheet = writer.sheets[sheet_name]

# chart = workbook.add_chart({'type': 'pie'})

# # Configure the series of the chart from the dataframe data.
# for col_num, value in enumerate(df.columns.values):
#     chart.add_series({
#         'name':       [sheet_name, 0, col_num],
#         'categories': [sheet_name, 1, 0, len(df), 0],
#         'values':     [sheet_name, 1, col_num, len(df), col_num],
#     })

# # Insert the chart into the worksheet.
# worksheet.insert_chart('D2', chart)

# # Close the Pandas Excel writer and output the Excel file.
# writer.save()


/var/folders/pl/v8kzknmx44v8w9mzp7wds7z40000gn/T/ipykernel_9235/813905487.py:34: FutureWarning:

save is not part of the public API, usage can give unexpected results and will be removed in a future version



In [ ]:
##get those contains Fabricación de productos elaborados de metal

exportacionallmetal=exportacionall[exportacionall['Descripción']=='Fabricación de productos elaborados de metal, excepto maquinaria y equipo']

exportacionallmetal=exportacionallmetal.groupby(['REGION_DE_ORIGEN']).agg({'VALOR_FOB_USD': 'sum'}).sort_values(by='VALOR_FOB_USD', ascending=False)
exportacionallmetal=exportacionallmetal.reset_index()

#otros
exportacionallmetal['REGION_DE_ORIGEN'] = exportacionallmetal['REGION_DE_ORIGEN'].apply(lambda x: 'Otros' if x not in exportacionallmetal[exportacionallmetal['VALOR_FOB_USD'] > exportacionallmetal['VALOR_FOB_USD'].sum()*0.01]['REGION_DE_ORIGEN'].values else x)

fig=px.pie(exportacionallmetal, values='VALOR_FOB_USD', names='REGION_DE_ORIGEN', title='Exportaciones Atlántico por CIIU para Fabricación de productos elaborados de metal, excepto maquinaria y equipo')
fig.show()

En cuanto el mercado internacional que abarca el departamento del Atlántico, podemos observar que el principal destino de las exportaciones del departamento es Estados Unidos, seguido Brasil y Mexico. En el caso de las importaciones, el principal origen de las mismas es Estados Unidos, seguido por China y Brasil.

In [ ]:
## get proportions of PAIS_DESTINO_FINAL from exportaciones to atlantico
exportacion=dd.read_csv('2022_exportacion.csv', dtype={'COD_MODALIDAD_PRECEDENTE': 'object','NIT_EXPORTADOR': 'object'}).compute()

exportacion=exportacion[exportacion['REGION_DE_ORIGEN']=='ATLANTICO']
exportacion=exportacion.groupby(['PAIS_DESTINO_FINAL']).agg({'VALOR_FOB_USD': 'sum'}).sort_values(by='VALOR_FOB_USD', ascending=False)
exportacion=exportacion.reset_index()

#otros are those that are less than the 0.0003% of the total
exportacion['PAIS_DESTINO_FINAL'] = exportacion['PAIS_DESTINO_FINAL'].apply(lambda x: 'Otros' if x not in exportacion[exportacion['VALOR_FOB_USD'] > exportacion['VALOR_FOB_USD'].sum()*0.01]['PAIS_DESTINO_FINAL'].values else x)


fig=px.pie(exportacion, values='VALOR_FOB_USD', names='PAIS_DESTINO_FINAL', title='Exportaciones Atlántico por País')
fig.show()


In [78]:
# #recreate the graph in excel

# # columns are PAIS_DESTINO_FINAL	VALOR_FOB_USD

# exportacion2=exportacion


# # Create a Pandas Excel writer using XlsxWriter as the engine.
# excel_file = 'pie_chartexport.xlsx'
# sheet_name = 'Sheet1'

# df=exportacion2

# writer = pd.ExcelWriter(excel_file, engine='xlsxwriter')
# df.to_excel(writer, sheet_name=sheet_name)

# workbook = writer.book
# worksheet = writer.sheets[sheet_name]

# chart = workbook.add_chart({'type': 'pie'})

# # Configure the series of the chart from the dataframe data.
# for col_num, value in enumerate(df.columns.values):
#     chart.add_series({
#         'name':       [sheet_name, 0, col_num],
#         'categories': [sheet_name, 1, 0, len(df), 0],
#         'values':     [sheet_name, 1, col_num, len(df), col_num],
#     })

# # Insert the chart into the worksheet.
# worksheet.insert_chart('D2', chart)

# # Close the Pandas Excel writer and output the Excel file.
# writer.save()



/var/folders/pl/v8kzknmx44v8w9mzp7wds7z40000gn/T/ipykernel_10911/460022180.py:34: FutureWarning:

save is not part of the public API, usage can give unexpected results and will be removed in a future version



In [ ]:
importacionesaa=importacion[importacion['DEPARTAMENTO_DESTINO']=='ATLANTICO']

importacionesaa=importacionesaa.groupby(['PAIS_ORIGEN']).agg({'VALOR_FOB_USD': 'sum'}).sort_values(by='VALOR_FOB_USD', ascending=False)
importacionesaa=importacionesaa.sort_values(by='VALOR_FOB_USD', ascending=False)
#otros


In [ ]:
importacionesaa=importacionesaa.reset_index()
#otros
importacionesaa['PAIS_ORIGEN'] = importacionesaa['PAIS_ORIGEN'].apply(lambda x: 'Otros' if x not in importacionesaa[importacionesaa['VALOR_FOB_USD'] > importacionesaa['VALOR_FOB_USD'].sum()*0.01]['PAIS_ORIGEN'].values else x)

In [ ]:
fig=px.pie(importacionesaa, values='VALOR_FOB_USD', names='PAIS_ORIGEN', title='Importaciones Atlántico por País')
fig.show()



In [ ]:
importacionesaa.columns

In [ ]:
# # #recreate the graph in excel

# # # columns are PAIS_ORIGEN', 'VALOR_FOB_USD

# importacionesaa2=importacionesaa


# # # Create a Pandas Excel writer using XlsxWriter as the engine.
# excel_file = 'pie_chartimportaciones.xlsx'
# sheet_name = 'Sheet1'

# df=importacionesaa2

# writer = pd.ExcelWriter(excel_file, engine='xlsxwriter')

# df.to_excel(writer, sheet_name=sheet_name)

# workbook = writer.book

# worksheet = writer.sheets[sheet_name]

# chart = workbook.add_chart({'type': 'pie'})

# # Configure the series of the chart from the dataframe data.

# for col_num, value in enumerate(df.columns.values):
    
#         chart.add_series({
    
#             'name':       [sheet_name, 0, col_num],
    
#             'categories': [sheet_name, 1, 0, len(df), 0],
    
#             'values':     [sheet_name, 1, col_num, len(df), col_num],
    
#         })

# # Insert the chart into the worksheet.

# worksheet.insert_chart('D2', chart)

# # Close the Pandas Excel writer and output the Excel file.

# writer.save()





/var/folders/pl/v8kzknmx44v8w9mzp7wds7z40000gn/T/ipykernel_10911/3131368426.py:44: FutureWarning:

save is not part of the public API, usage can give unexpected results and will be removed in a future version



## 2. Funcionamiento de Cuentas Nacionales del Atlántico

Revisando el historico de PIB departamental, encontramos que el Atlatnico ha tenido un crecimiento constante en los ultimos años, siendo la 6ta economia mas grande del pais en 2022. 


In [ ]:
# #recreate the graph in excel

# # columns are REGION_DE_ORIGEN	VALOR_FOB_USD

# exportacionallmetal2=exportacionallmetal


# # Create a Pandas Excel writer using XlsxWriter as the engine.
# excel_file = 'pie_chart2.xlsx'
# sheet_name = 'Sheet1'

# df=exportacionallmetal2

# writer = pd.ExcelWriter(excel_file, engine='xlsxwriter')
# df.to_excel(writer, sheet_name=sheet_name)

# workbook = writer.book
# worksheet = writer.sheets[sheet_name]

# chart = workbook.add_chart({'type': 'pie'})

# # Configure the series of the chart from the dataframe data.
# for col_num, value in enumerate(df.columns.values):
#     chart.add_series({
#         'name':       [sheet_name, 0, col_num],
#         'categories': [sheet_name, 1, 0, len(df), 0],
#         'values':     [sheet_name, 1, col_num, len(df), col_num],
#     })
    

# # Insert the chart into the worksheet.
# worksheet.insert_chart('D2', chart)

# # Close the Pandas Excel writer and output the Excel file.
# writer.save()



/var/folders/pl/v8kzknmx44v8w9mzp7wds7z40000gn/T/ipykernel_9235/2795901221.py:35: FutureWarning:

save is not part of the public API, usage can give unexpected results and will be removed in a future version



In [ ]:
pib=pd.read_csv("/Users/lasagna/Library/CloudStorage/OneDrive-UniversidaddelNorte/Probarranquilla Projects/PIB proyectado/chart.csv")
#from col 2021pr to the end, delete the cols
colnum2021pr=pib.columns.get_loc("2021pr")
pib=pib.iloc[:, :colnum2021pr+1]
pib.rename(columns={'2021pr': '2021'}, inplace=True)

# plot all DEPARTAMENTOS 
pib=pib.melt(id_vars=['DEPARTAMENTOS'], var_name='Año', value_name='PIB')
# change 2020p to 2020
pib['Año']=pib['Año'].str.replace('p', '')
pib['Año']=pib['Año'].astype(int)
# delete , 
pib['PIB']=pib['PIB'].str.replace(',', '')
pib['PIB']=pib['PIB'].astype(float)

# get only departamentos with more than 1% of the total

# delete COLOMBIA from DEPARTAMENTOS
pib=pib[pib['DEPARTAMENTOS']!='COLOMBIA']
# delete Bogotá D.C. from DEPARTAMENTOS
pib=pib[pib['DEPARTAMENTOS']!='Bogotá D.C.']

#others less than 20000
pib['DEPARTAMENTOS'] = pib['DEPARTAMENTOS'].apply(lambda x: 'Otros' if x not in pib[pib['PIB'] > 20000]['DEPARTAMENTOS'].values else x)
#delete otros
pib=pib[pib['DEPARTAMENTOS']!='Otros']

#plot
fig=px.line(pib, x='Año', y='PIB', color='DEPARTAMENTOS', title='PIB Real por departamento en Miles de millones de pesos')
fig.show()



In [ ]:
# #recreate the graph in excel

# # columns are DEPARTAMENTOS	Año	PIB

# pib2=pib

# # convert DEPARTAMENTOS TO COLUMN
# pib2=pib2.pivot(index='Año', columns='DEPARTAMENTOS', values='PIB')
# pib2=pib2.reset_index()



# # Create a Pandas Excel writer using XlsxWriter as the engine.
# excel_file = 'line_chartbydepartment.xlsx'
# sheet_name = 'Sheet1'

# df=pib2

# writer = pd.ExcelWriter(excel_file, engine='xlsxwriter')

# df.to_excel(writer, sheet_name=sheet_name)

# workbook = writer.book

# worksheet = writer.sheets[sheet_name]

# chart = workbook.add_chart({'type': 'line'})

# # Configure the series of the chart from the dataframe data.

# for col_num, value in enumerate(df.columns.values):
#     if col_num > 0:
#         chart.add_series({
#             'name':     [sheet_name, 0, col_num],
#             'categories': [sheet_name, 1, 0, len(df), 0],
#             'values':     [sheet_name, 1, col_num, len(df), col_num],
            
#         })

# # Insert the chart into the worksheet.
# worksheet.insert_chart('D2', chart)

# # Close the Pandas Excel writer and output the Excel file.
# writer.save()





/var/folders/pl/v8kzknmx44v8w9mzp7wds7z40000gn/T/ipykernel_9235/1356142412.py:44: FutureWarning:

save is not part of the public API, usage can give unexpected results and will be removed in a future version



In [ ]:
# JUST FOR 2021, get %
pib2021=pib[pib['Año']==2021]
pib2021['PIB']=pib2021['PIB']/pib2021['PIB'].sum()*100
pib2021=pib2021.sort_values(by='PIB', ascending=False)
pib2021
fig=px.pie(pib2021, values='PIB', names='DEPARTAMENTOS', title='PIB Real por departamento')
fig.show()


/var/folders/pl/v8kzknmx44v8w9mzp7wds7z40000gn/T/ipykernel_10911/702171108.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
# # Create a Pandas Excel writer using XlsxWriter as the engine.
# excel_file = 'pie_chart2021.xlsx'
# sheet_name = 'Sheet1'

# df=pib2021

# writer = pd.ExcelWriter(excel_file, engine='xlsxwriter')

# df.to_excel(writer, sheet_name=sheet_name)

# workbook = writer.book

# worksheet = writer.sheets[sheet_name]

# chart = workbook.add_chart({'type': 'pie'})

# # Configure the series of the chart from the dataframe data.

# for col_num, value in enumerate(df.columns.values):
#     if col_num > 0:
#         chart.add_series({
#             'name':     [sheet_name, 0, col_num],
#             'categories': [sheet_name, 1, 0, len(df), 0],
#             'values':     [sheet_name, 1, col_num, len(df), col_num],
            
#         })

# # Insert the chart into the worksheet.

# worksheet.insert_chart('D2', chart)

# # Close the Pandas Excel writer and output the Excel file.

# writer.save()


/var/folders/pl/v8kzknmx44v8w9mzp7wds7z40000gn/T/ipykernel_9235/636664010.py:34: FutureWarning:

save is not part of the public API, usage can give unexpected results and will be removed in a future version



Segun Proyecciones realizadas por ProBarranquilla, se espera que el PIB departamental haya alcanzado a 42.9 miles de millones de pesos en 2022, lo que representa un crecimiento de casi 5% con respecto al 2021.


In [ ]:
## get proyectado
pibproyectado=pd.read_csv("/Users/lasagna/Library/CloudStorage/OneDrive-UniversidaddelNorte/Probarranquilla Projects/PIB proyectado/Proyeccion.csv")
pibproyectado[["DEPARTAMENTOS", "Atlántico"]]
pibproyectado.rename(columns={'DEPARTAMENTOS': 'Año'}, inplace=True)

# convert to change the values
pibproyectado["Atlántico"]=pibproyectado["Atlántico"].str.replace(',', '')
pibproyectado["Atlántico"]=pibproyectado["Atlántico"].astype(float)

#plot
fig=px.line(pibproyectado, x='Año', y='Atlántico', title='PIB Proyectado del Atlántico en Miles de millones de pesos')
fig.show()



In [ ]:
# # Create a Pandas Excel writer using XlsxWriter as the engine.
# excel_file = 'line_chartproyectado.xlsx'
# sheet_name = 'Sheet1'

# df=pibproyectado

# writer = pd.ExcelWriter(excel_file, engine='xlsxwriter')

# df.to_excel(writer, sheet_name=sheet_name)

# workbook = writer.book

# worksheet = writer.sheets[sheet_name]

# chart = workbook.add_chart({'type': 'line'})

# # Configure the series of the chart from the dataframe data.

# for col_num, value in enumerate(df.columns.values):
#     if col_num > 0:
#         chart.add_series({
#             'name':     [sheet_name, 0, col_num],
#             'categories': [sheet_name, 1, 0, len(df), 0],
#             'values':     [sheet_name, 1, col_num, len(df), col_num],
            
#         })

# # Insert the chart into the worksheet.
# worksheet.insert_chart('D2', chart)

# # Close the Pandas Excel writer and output the Excel file.
# writer.save()


/var/folders/pl/v8kzknmx44v8w9mzp7wds7z40000gn/T/ipykernel_9235/2938075198.py:32: FutureWarning:

save is not part of the public API, usage can give unexpected results and will be removed in a future version

